# LLM Classification Fine Tunning

In [9]:
# Import necessary libraries
import pandas as pd  # For data manipulation
import torch  # PyTorch deep learning framework
from transformers import (DebertaTokenizer, DebertaForSequenceClassification,
                         Trainer, TrainingArguments)  # Hugging Face transformers

In [ ]:
# Importing and storing datasets as whole (train.csv + test.csv) and seperately.

train_dataset =  pd.read_csv(r'DataSet\llm-classification-finetuning\train.csv')
test_dataset =  pd.read_csv(r'DataSet\llm-classification-finetuning\train.csv')
original_dataset = pd.concat([train_dataset, test_dataset])

In [8]:
# Checking 'columns' present in full dataset
print("\nColumn names:\n",original_dataset.columns)


Column names:
 Index(['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b',
       'winner_model_a', 'winner_model_b', 'winner_tie'],
      dtype='object')


In [11]:
# Converting text to numbers that models can understand using Tokenizers.
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')

In [17]:
# Preprocessing Dataset(each row) by removing special characters and lowercasing everything.
def preprocess(row):
   """
   Combines prompt and both responses into a single text string,
   then tokenizes it for the model
    
   row: One row of our DataFrame containing prompt, response_a, response_b
   Returns: Tokenized output ready for the model
   """

   text = (
      f"Prompt: {row['prompt']} [SEP] "
      f"Response A: {row['response_a']} [SEP] "
      f"Response B: {row['response_b']}"
      )
   # truncation = True: #Cutting/Limiting text under max_length
   # max_length=512: Maximum sequence length DeBERTa can handle
   return tokenizer(text, truncation=True, max_length=512)

# Applying preprocessing to all rows in training and test data
# This converts our text data into numerical representations (tensors)
train_encoder = train_dataset.apply(preprocess, axis=1).tolist()
test_encoder = test_dataset.apply(preprocess, axis=1).tolist()

In [20]:
# Initializing the DeBERTa model for sequence classification
model = DebertaForSequenceClassification.from_pretrained(
    "microsoft/deberta-base",  # Base pre-trained model
    num_labels=3,  # 3 output classes: model_a wins, model_b wins, tie
    problem_type="single_label_classification"  # Each input has exactly one label
)

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [24]:
# Setting up training configs
training_args = TrainingArguments(
   output_dir = './results',
   num_train_epochs = 3,
   per_device_train_batch_size=8,
   evaluation_strategy="epoch",
   save_strategy="epoch",
   logging_steps=100,
   learning_rate=2e-5,
   warmup_ratio=0.1,
   weight_decay=0.01,
   fp16=True
)

c:\Users\Aditya King\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Creating -> Trainer object to handle training.
trainer = Trainer(
   model=model, # DeBERTa model
   args=training_args, # Training configuration
   train_dataset=train_encoder, # Training data
   eval_dataset=val_encoder # Validation data (Splitted from train)
   # compute_metrics=compute_metrics  # For tracking metrics
)

In [ ]:
# Training!

print("Training Started...\n")
trainer.train()
print("\nTraining Completed!\n")

In [ ]:
# Generating predictions on test data
test_predictions = trainer.predict(test_encoder) # Getting raw model outputs

# Converting raw outputs to probabilities using softmax because softmax ensures probabilities sum to 1
probs = torch.softmax(torch.tensor(test_predicitons), dim=1).numpy()


In [ ]:
# Creating submission file in required format
sub = pd.DataFrame({
   "id":test["id"], # Keeping original IDs
   "winner_model_a":probs[:,0], # Probability of model A winning
   "winner_model_b":probs[:,1], # Probability of model B winning
   "winner_model_tie":probs[:,2] # Probability of tie
})

In [ ]:
# Saving predictions to CSV
sub.to_csv("submission.csv", index=False)